In [2]:
import time
import pandas as pd 
from colorthief import ColorThief
from joblib import Parallel, delayed 
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier

from Hyprparam_optimized.searcher import GridSearch, RandomSearch
from Hyprparam_optimized.parallel_process import ParallelExecutor
from Hyprparam_optimized.logger import log_to_csv, log_to_json

In [3]:
df = pd.read_csv(r"train_and_test2.csv")

df.tail()

,Passengerid,Age,Fare,Sex,sibsp,zero,zero.1,zero.2,zero.3,zero.4,...,zero.12,zero.13,zero.14,Pclass,zero.15,zero.16,Embarked,zero.17,zero.18,2urvived
1304,1305,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1305,1306,39.0,108.9000,1,0,0,0,0,0,0,...,0,0,0,1,0,0,0.0,0,0,0
1306,1307,38.5,7.2500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1307,1308,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,3,0,0,2.0,0,0,0
1308,1309,28.0,22.3583,0,1,0,0,0,0,0,...,0,0,0,3,0,0,0.0,0,0,0


In [4]:
df = df.drop(columns=['zero','zero.4','zero.15','zero.17','zero.18'])
df

,Passengerid,Age,Fare,Sex,sibsp,zero.1,zero.2,zero.3,zero.5,zero.6,...,zero.9,zero.10,zero.11,zero.12,zero.13,zero.14,Pclass,zero.16,Embarked,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0.0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2.0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,0
1305,1306,39.0,108.9000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0.0,0
1306,1307,38.5,7.2500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,0
1307,1308,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,0


In [5]:
df = df.drop_duplicates()
df = df.dropna()

In [6]:
df

,Passengerid,Age,Fare,Sex,sibsp,zero.1,zero.2,zero.3,zero.5,zero.6,...,zero.9,zero.10,zero.11,zero.12,zero.13,zero.14,Pclass,zero.16,Embarked,2urvived
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0.0,1
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,1
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,2.0,1
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,0
1305,1306,39.0,108.9000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0.0,0
1306,1307,38.5,7.2500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,0
1307,1308,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,2.0,0


In [7]:
y = df['2urvived']
x = df.drop(columns='2urvived')
x

,Passengerid,Age,Fare,Sex,sibsp,zero.1,zero.2,zero.3,zero.5,zero.6,...,zero.8,zero.9,zero.10,zero.11,zero.12,zero.13,zero.14,Pclass,zero.16,Embarked
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.0
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2.0
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0
1305,1306,39.0,108.9000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.0
1306,1307,38.5,7.2500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0
1307,1308,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0


In [8]:
x

,Passengerid,Age,Fare,Sex,sibsp,zero.1,zero.2,zero.3,zero.5,zero.6,...,zero.8,zero.9,zero.10,zero.11,zero.12,zero.13,zero.14,Pclass,zero.16,Embarked
0,1,22.0,7.2500,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0
1,2,38.0,71.2833,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.0
2,3,26.0,7.9250,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0
3,4,35.0,53.1000,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2.0
4,5,35.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,1305,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0
1305,1306,39.0,108.9000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.0
1306,1307,38.5,7.2500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0
1307,1308,28.0,8.0500,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0


In [9]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [10]:
x_train

,Passengerid,Age,Fare,Sex,sibsp,zero.1,zero.2,zero.3,zero.5,zero.6,...,zero.8,zero.9,zero.10,zero.11,zero.12,zero.13,zero.14,Pclass,zero.16,Embarked
1294,1295,17.0,47.1000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2.0
544,545,50.0,106.4250,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0.0
290,291,26.0,78.8500,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2.0
10,11,4.0,16.7000,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,2.0
148,149,36.5,26.0000,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1097,1098,35.0,7.7500,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,3,0,1.0
1132,1133,45.0,30.0000,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,2.0
1296,1297,20.0,13.8625,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,0.0
862,863,48.0,25.9292,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,2.0


In [11]:
pipeline = Pipeline([
    ("scaler", StandardScaler()),
    ("clf", GradientBoostingClassifier())
])

In [12]:
param_space = {
    "clf__n_estimators": [200,300, 500],
    "clf__learning_rate": [0.03, 0.05, 0.1],
    "clf__max_depth": [2, 3, 4],
    "clf__min_samples_split": [2, 5],
    "clf__min_samples_leaf": [1, 2],
}


In [13]:
t1 = time.time()
grid = GridSearch(param_space)
grid_params = grid.generate()

executor = ParallelExecutor(n_jobs=-1)
grid_results = executor.run(
    estimator=pipeline,
    X=x_train,
    y=y_train,
    param_list=grid_params,
    cv=5,
    scoring="accuracy"
)
t2 = time.time()
log_to_csv(grid_results, "sample_logs/grid_results.csv")
print(t2-t1)

47.78624749183655


In [14]:
random_search = RandomSearch(param_space, n_iter=10, random_state=42)
random_params = random_search.generate()

random_results = executor.run(
    estimator=pipeline,
    X=x_train,
    y=y_train,
    param_list=random_params,
    cv=5,
    scoring="accuracy"
)

log_to_json(random_results, "sample_logs/random_results.json")


In [15]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2],
}
gbc = GradientBoostingClassifier(random_state=42)

In [16]:
t1 = time.time()
grid_search = GridSearchCV(
    estimator=gbc, 
    param_grid=param_grid, 
    scoring='accuracy', 
    cv=5, 
    n_jobs=None  
)
grid_search.fit(x_train,y_train)
t2 = time.time()
print(t2-t1)

78.50023055076599


In [17]:
print("Number of parameter combinations:", len(grid_params))
print("Total model fits:", len(grid_params) * 5)  # cv=5


Number of parameter combinations: 108
Total model fits: 540
